# Basique tutorial ComSeg

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
import sys
import numpy as np

In [2]:
import matplotlib
matplotlib.use('Qt5Agg')
import sys
import numpy as np
sys.path  
sys.path +=[
           '/home/tom/anaconda3/envs/comseg_v0/lib/python3.8/site-packages'] ## add conda env

sys.path

['/home/tom/Bureau/phd/simulation/ComSeg_pkg/tutorial',
 '/home/tom/anaconda3/lib/python38.zip',
 '/home/tom/anaconda3/lib/python3.8',
 '/home/tom/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/home/tom/.local/lib/python3.8/site-packages',
 '/home/tom/anaconda3/lib/python3.8/site-packages',
 '/home/tom/Bureau/phd/autofish_pckg',
 '/home/tom/anaconda3/envs/comseg_v0/lib/python3.8/site-packages']

# dataset initialization

In [3]:
import comseg
from comseg import dataset
import importlib
importlib.reload(dataset)

<module 'comseg.dataset' from '/home/tom/anaconda3/envs/comseg_v0/lib/python3.8/site-packages/comseg/dataset.py'>

In [4]:

### List of genes to study
selected_genes = ['Atp6v0d2', 'Abcg1','Rtkn2',  'Igfbp2','Cxcl15','Cd79a','Ms4a2', 'Fcer1a']
 #                    'Ccdc153','Scgb3a2', 'Scgb1a1', 'Cst3','Cdh5', 'Clec14a','Inmt', ]
  #                   'Pcolce2','C1qc', 'C1qa', 'C1qb','Upk3b','Ifitm6','Plac8','Ms4a4b', 'Ccl5', 'Hcst',
   #                  'Gzma', 'Ncr1','S100a9','Mmrn1', 'Acta2','Myh11', 'Cd3g', 'Cd3d']

dataset = comseg.dataset.ComSegDataset(selected_genes = selected_genes,
    path_dataset_folder = "/media/tom/T7/simulation/test_set/small_df",
                path_to_mask_prior = "/media/tom/T7/simulation/test_set/mask")
dataset.add_prior_from_mask()

In [6]:

dataset.compute_edge_weight(dico_scale={"x": 0.103, 'y': 0.103, "z": 0.3},  # in micrometer
        images_subset = None,
        mode="nearest_nn_radius",
        n_neighbors=25,
        radius=1,  # in micormeter
        distance="pearson",
        per_images=False,
        sampling=False,
        sampling_size=100000)

TypeError: compute_edge_weight() got an unexpected keyword argument 'dico_scale'

# CREATE RNA GRAPH

In [15]:
import comseg

from comseg import model, dictionary
import importlib
importlib.reload(comseg)
importlib.reload(model)
importlib.reload(dictionary)

<module 'comseg.dictionary' from '/home/tom/Bureau/phd/simulation/ComSeg_pkg/comseg/dictionary.py'>

In [ ]:

Comsegdict = dictionary.ComSegDict()
for img_name  in dataset:
    comseg_m = model.ComSeg(selected_genes = dataset.selected_genes,
        df_spots_label = dataset[img_name],
                 dict_scale={"x": 0.103, 'y': 0.103, "z": 0.3})
    comseg_m.create_graph(dict_co_expression=dataset.dict_co_expression ,
                         )
    comseg_m.community_vector(
                             clustering_method="louvain_with_prior",
                             weights_name="weight",

                             prior_keys="prior",
                             seed=None,
                             super_node_prior_keys="prior",
                             confidence_level=1,
                             )
    Comsegdict[img_name] = comseg_m
    

computing knn
adding edges


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 84684/84684 [00:01<00:00, 66719.42it/s]


creation of  super node with prior
1
2655
468
171
105
37
26
21
20
20
12
9
4
True
322
1
93
9
229
1
12
217
1
217


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3346/3346 [00:00<00:00, 9349.04it/s]


In [ ]:
 img_name

In [ ]:
import anndata as ad
Comsegdict.concatenate_anndata()

# IN SITU CLUSTERING detail

In [ ]:
import comseg
from comseg import clustering
import importlib
importlib.reload(comseg)
importlib.reload(clustering)
from comseg import clustering

In [ ]:
insitu_clustering  = clustering.InSituClustering(anndata = Comsegdict.global_anndata,
                                                 selected_genes = dataset.selected_genes)

In [ ]:
norm_expression_vectors, param_sctransform = insitu_clustering.compute_normalization_parameters( )

In [ ]:
l = Comsegdict.global_anndata.var_names
l

In [ ]:
bool_index  = np.array(insitu_clustering.anndata.obs['nb_rna']) > 3
insitu_clustering.anndata[bool_index, :]

In [ ]:
insitu_clustering.anndata

In [16]:
anndata = insitu_clustering.cluster_rna_community(size_commu_min=3,
                              norm_vector = True,
                              ### parameter clustering
                              n_pcs = 3,
                              n_comps =3,
                              clustering_method ="leiden",
                              n_neighbors = 20,
                              resolution = 1,
                              n_clusters_kmeans = 4,
                              palette = None)

number of cluster 5


In [17]:
anndata

AnnData object with n_obs × n_vars = 408 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna', 'img_name', 'leiden'

In [18]:
insitu_clustering.cluster_centroid(
                      cluster_column_name = "leiden",
                       agregation_mode = "mean")

0
1
2
3
4


(array([[ 2.7499287 ,  1.9224914 ,  0.11273305, -0.19154239, -0.20656845,
         -0.38980484],
        [-1.0912926 , -0.7754209 , -0.5186857 , -0.5095918 ,  0.07091062,
         -0.63465077],
        [-0.4151709 , -0.43576032, -0.25616103, -0.2569109 , -0.2714332 ,
          0.6625279 ],
        [-0.8202157 , -0.68373597,  0.18360461,  2.2041764 ,  0.34213322,
         -0.6109246 ],
        [-0.7858103 , -0.54738176, -0.16928597, -0.19157282, -0.37327442,
          5.6902933 ]], dtype=float32),
 array(['0', '1', '2', '3', '4'], dtype='<U1'))

In [19]:
insitu_clustering.merge_cluster(nb_min_cluster = 0,
                      min_merge_correlation = 0.1,
                      cluster_column_name="leiden",
                      plot = False)

[2.   4.   0.22 2.  ]
[1.   5.   0.63 3.  ]
[3.   6.   0.85 4.  ]


# take the cluster and add it in the model 

In [29]:
insitu_clustering.anndata

AnnData object with n_obs × n_vars = 408 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna', 'img_name', 'leiden'

In [28]:
Comsegdict.global_anndata[Comsegdict.global_anndata.obs.img_name == "07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004"]

View of AnnData object with n_obs × n_vars = 217 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna', 'img_name', 'leiden'

In [24]:
Comsegdict

{'07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004': <comseg.model.ComSeg object at 0x7f51754d4f70>, '07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_006': <comseg.model.ComSeg object at 0x7f51031d9af0>, 'global_anndata': AnnData object with n_obs × n_vars = 408 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna', 'img_name', 'leiden'}

In [27]:
Comsegdict['07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004'].community_anndata

AnnData object with n_obs × n_vars = 217 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna'
    var: 'features'

In [32]:
np.unique(
    Comsegdict.global_anndata[
        Comsegdict.global_anndata.obs.img_name == "07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004"].obs.leiden)

array(['-1', '0', '1', '2', '3', '4'], dtype=object)

In [ ]:
 "07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004"
Comsegdict.global_anndata.obs[clustering_method].loc[self[img].community_anndata.obs.index].value

In [38]:
Comsegdict.global_anndata = insitu_clustering.anndata
clustering_method = 'leiden'
for img in Comsegdict:
    if str(type(Comsegdict[img])) != "<class 'comseg.model.ComSeg'>":
        continue
    print(img)
    ## get the cluster list and the community index

    cluster_id_list = list(Comsegdict.global_anndata[
        Comsegdict.global_anndata.obs.img_name == img].obs[clustering_method])
    community_index_list = list(Comsegdict.global_anndata[Comsegdict.global_anndata.obs.img_name == img].obs.index_commu)

    assert community_index_list == list(Comsegdict[img].community_anndata.obs['index_commu'])

    Comsegdict[img].community_anndata.obs[clustering_method] = cluster_id_list
    
    

07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_004
07_CtrlNI_Pdgfra-Cy3_Serpine1-Cy5_006


In [37]:
Comsegdict.global_anndata[Comsegdict.global_anndata.obs.img_name == img]

View of AnnData object with n_obs × n_vars = 217 × 8
    obs: 'list_coord', 'node_index', 'prior', 'index_commu', 'nb_rna', 'img_name', 'leiden'

## add centroid to model 


## classify it 
## apply dikstra 
## 

In [ ]:
scrna_unique_clusters

In [ ]:
np.array(insitu_clustering.scrna_unique_clusters).astype(int)

In [ ]:
 insitu_clustering.scrna_centroids.shape